# Simple Solution SymPy Code Generation

This notebook is based on Mark Ghiorso's Simple-Solution notebook in ThermoEngine for generating an asymmetric regular solution model as Modified by Lucy Tweed for an Asymmetric Binary solution in the Fo-SiO2 system

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import hashlib
import warnings
import time
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../..')
SPUD_DIR = HOME_DIR+'/phases'
try:
    os.makedirs(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'fo_sio2/notebooks/Generate_asymmetric_binary_fosi_solution.ipynb'

## Simple Solution Properties - Structure of the Equations
There are three terms:
- Terms describing standard state contributions
- Terms describing the configurational entropy of solution
- Terms describing the excess enthalpy of solution  

Assumptions:
- There are $c$ components in the system
- There are as many endmember species, $s$, as there are components
- The configurational entropy is described as a simple $x_i log(x_i)$ sum
- The excess enthalpy is described using an asymmetric regular solution described below
- Ternary interaction terms are not included in a binary system

## Number of solution components
This notebook illustrates a two component solution

In [ ]:
c = 2

## Create a simple solution model
... with the specified number of endmember thermodynamic components

In [ ]:
model = coder.SimpleSolnModel.from_type(nc=c)

## Retrieve primary compositional variables
- $n$ is a vector of mole numbers of each component  
- $n_T$ is the total number of moles in the solution
### and construct a derived mole fraction variable
- $X$ is a vector of mole fractions of components in the system

In [ ]:
n = model.n
nT = model.nT
X = n/nT
n, nT, X

## Retrieve the temperature, pressure, and standard state chemical potentials
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $\mu$ in Joules

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Pr = model.get_symbol_for_pr()
mu = model.mu
T,P,mu,Pr

Check model dictionary

In [ ]:
model.model_dict

## Define the standard state contribution to solution properties

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

## Define configurational entropy and configurational Gibbs free energy

In [ ]:
S_config,R = sym.symbols('S_config R')
S_config = 0
for i in range(0,c):
    S_config += X[i]*sym.log(X[i])
S_config *= -R*nT
S_config

In [ ]:
G_config = sym.simplify(-T*S_config)
G_config

## Parameterize the excess free energy
This is based on Lucy Tweed's pyezthermo model for G_excess of mixing

where $W$, $dWdT$ and $dWdP$  are two-vectors (i.e.)
$$
    W = \begin{bmatrix} W_1\\ W_2\end{bmatrix}\quad dWdT = \begin{bmatrix} dWdT_1\\ dWdT_2\end{bmatrix}\quad dWdP = \begin{bmatrix} dWdP_1\\ dWdP_2\end{bmatrix}
$$
with total Margules parameters

$$
    W_{tot} = W + T*dWdT + (P-Pr)*dWdT
$$
and
$$
    G_{excess} = (n\cdot W_{tot})X_1X_2
$$

In [ ]:
params = ['R']
units = ['J/mol/K']
symparam = (R,)
W0 = sym.Matrix
G_excess = sym.symbols('G_excess')
G_excess = 0
for i in range(1,c+1):
        param = 'W{}'.format(i); params.append(param); units.append('J/m')
        h_term = sym.symbols(param); symparam += (h_term,)
        param = 'dWdT{}'.format(i); params.append(param); units.append('J/K-m')
        dT_term = sym.symbols(param); symparam += (dT_term,)
        param = 'dWdP{}'.format(i); params.append(param); units.append('J/bar-m')
        dP_term = sym.symbols(param); symparam += (dP_term,)
        w_term = h_term + T*dT_term + (P-Pr)*dP_term
        G_excess += w_term*n[i-1]
G_excess *= X[0]*X[1]
G_excess

In [ ]:
print(params)
print(units)
symparam

## Define the Gibbs free energy of solution

In [ ]:
G = G_ss + G_config + G_excess
G

## Add the Gibbs free energy of solution to the model

In [ ]:
model.add_potential_to_model('G',G,list(zip(params, units, symparam)))

### let's inspect the dictionary and unset parameters

In [ ]:
model.model_dict

In [ ]:
values_dict = model.get_values()
values_dict

... assign a formula string for code generation  
... assign a conversion string to map element concentrations to moles of end members

In [ ]:
values_dict.update(dict(name='Liquid',
                        abbrev='Lq',
                        reference=reference,
                        formula_string='Mg[Mg]Si[Si]O[O]',
                        conversion_string=['[0]=([Si] - [Mg]/2.)/2.', '[1]=0.5*[Mg]' ],
                        test_string = ['[0] > 0.0', '[1] > 0.0' ],
                        endmembers=[ 'Quartz4_liquid','Forsterite_xmelts'],
                        R=8.31446261815324,
                        W1=35168., W2= -56504.,
                        dWdT1=0., dWdT2= 0.,
                        # original
                        dWdP1=0.7959, dWdP2= -1.8783
                        # Tweed 2021, thesis
                        #dWdP1=0.7059, dWdP2= -1.8783
                       ))
values_dict

In [ ]:
model.set_values(values_dict)
model.to_xml(path=SPUD_DIR)